# SageMaker JumpStart - invoke time series forecasting endpoint

This notebook demonstrates how to attach a predictor to an existing endpoint name and invoke the endpoint with example payloads.

In [2]:
from sagemaker.predictor import retrieve_default

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Retrieve a predictor from your deployed endpoint name.

In [3]:
endpoint_name = "jumpstart-dft-autogluon-forecasting-20240814-120208"
predictor = retrieve_default(endpoint_name)

Now query your endpoint with example payloads.

In [4]:
# Single time series
payload = {
    "inputs": [
        {"target": [0.0, 4.0, 5.0, 1.5, -3.0, -5.0, -3.0, 1.5, 5.0, 4.0, 0.0, -4.0, -5.0, -1.5, 3.0, 5.0, 3.0, -1.5, -5.0, -4.0]},
    ],
    "parameters": {
        "prediction_length": 10
    }
}
response = predictor.predict(payload)
print(response)

{'predictions': [{'mean': [-0.012903248891234398, 4.083284854888916, 4.222873687744141, 1.6410558223724365, -2.3331379890441895, -4.377713203430176, -3.053372383117676, 0.533724308013916, 3.6586508750915527, 3.4521987438201904], '0.1': [-1.5178885817527772, 1.9753665566444398, 3.8920821189880375, -0.434017652273178, -3.8334312438964844, -5.051026821136475, -4.121994161605834, -1.0439883112907409, 1.3513196468353272, 1.8439882755279542], '0.5': [4.470348358154297e-08, 4.997067928314209, 4.997067928314209, 1.747800588607788, -2.5689151287078857, -4.422287702560425, -3.014662742614746, 4.470348358154297e-08, 4.598240852355957, 4.011730194091797], '0.9': [1.5014662742614746, 4.997067928314209, 5.046334743499757, 3.1038123130798354, -1.287976598739622, -3.197653985023498, -1.4146627843379966, 2.4328446865081794, 4.997067928314209, 4.997067928314209]}]}


In [5]:
# Multiple time series with "item_id" and "start" fields
payload = {
    "inputs": [
        {"target": [1.0, 2.0, 3.0, 2.0, 0.5, 2.0, 3.0, 2.0, 1.0], "item_id": "product_A", "start": "2024-01-01T01:00:00"},
        {"target": [5.4, 3.0, 3.0, 2.0, 1.5, 2.0, -1.0], "item_id": "product_B", "start": "2024-02-02T03:00:00"},
    ],
    "parameters": {
        "prediction_length": 5,
        "freq": "1h",
        "quantile_levels": [0.05, 0.5, 0.95],
        "num_samples": 30,
        "batch_size": 2,
    }
}
response = predictor.predict(payload)
print(response)

{'predictions': [{'mean': [1.797491077137076, 2.1308243821582034, 2.114695367956948, 2.180555590262843, 1.981182830278865], '0.05': [0.22379034452606, 0.22379034452606, 2.561137080192566e-08, 2.561137080192566e-08, 2.561137080192566e-08], '0.5': [0.9946236908435822, 1.5053763290246327, 2.002688229084015, 2.002688229084015, 2.002688229084015], '0.95': [4.496639867623647, 5.764784934123355, 4.496639867623647, 4.552419428030647, 4.00537645816803], 'item_id': 'product_A', 'start': '2024-01-01T10:00:00'}, {'mean': [0.8686287121284888, 1.2254538615464263, 1.5941523822992165, 1.5729053637194463, 1.749755651159668], '0.05': [-1.20076980672836, -0.6542836726138351, -0.5905425620994523, -0.6439726184358666, -1.394805270477936], '0.5': [1.246700954906078, 1.4997906339033713, 1.649769621084912, 1.4997906339033713, 2.0059699156891617], '0.95': [2.5524559916944765, 2.9995809629715575, 3.9931924675067307, 3.7654117221152066, 4.558425991213315], 'item_id': 'product_B', 'start': '2024-02-02T10:00:00'}]

The payload must be structured as follows.
* **inputs** (required): List with at most 64 time series that need to be forecasted. Each time series is represented by a dictionary with the following keys:
    * **target** (required): List of observed numeric time series values. 
        - It is recommended that each time series contains at least 30 observations.
        - If any time series contains fewer than 5 observations, an error will be raised.
    * **item_id**: String that uniquely identifies each time series. 
        - If provided, the ID must be unique for each time series.
        - If provided, then the endpoint response will also include the **item_id** field for each forecast.
    * **start**: Timestamp of the first time series observation in ISO format (`YYYY-MM-DD` or `YYYY-MM-DDThh:mm:ss`). 
        - If **start** field is provided, then **freq** must also be provided as part of **parameters**.
        - If provided, then the endpoint response will also include the **start** field indicating the first timestamp of each forecast.
* **parameters**: Optional parameters to configure the model.
    * **prediction_length**: Integer corresponding to the number of future time series values that need to be predicted. 
        - Recommended to keep prediction_length <= 64 since larger values will result in inaccurate quantile forecasts. Values above 1000 will raise an error.
    * **quantile_levels**: List of floats in range (0, 1) specifying which quantiles should should be included in the probabilistic forecast. Defaults to `[0.1, 0.5, 0.9]`. 
    * **freq**: Frequency of the time series observations in [pandas-compatible format](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases). For example, `1h` for hourly data or `2W` for bi-weekly data. 
        - If **freq** is provided, then **start** must also be provided for each time series in **inputs**.
    * **num_samples**: Number of sample trajectories generated by the Chronos model during inference. Larger values may improve accuracy but increase memory consumption and slow down inference. Defaults to `20`.
    * **batch_size**: Number of time series processed in parallel by the model. Larger values speed up inference but may lead to out of memory errors.

All keys not marked with (required) are optional.

The endpoint response contains the probabilistic (quantile) forecast for each time series included in the request.